# Intro to Flask: Lesson 3

## Forms

We will now look into ways to create forms using Flask. First, we'll use HTML tags to make forms and connect our `app.py` to those elements in an extended example. Our app will include:

- A home page `index.html` at `/`
- A navigation bar (navbar) with a link to the home page
- A survey page `signup.html` at `/signupform` for users to enter their name
- A thank you page `thankyou.html` at `/thank_you` thanking users after competing the survey
- A custom 404 error page `404.html`

Just like our previous examples, `app.py` will be at the root of our project folder, and the following views will be in a folder named `templates/`:

- `base.html`
- `index.html`
- `signup.html`
- `thankyou.html`
- `404.html`

Below is the starting code for each of our project files. If you install packages (flask, etc), remember to install them inside of your virtual environment (venv).

In [1]:
# app.py

# request is another module to use on the thankyou page
from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/signup')
def signup():
    return render_template('signup.html')

@app.route('/thank_you')
def thank_you():
    first = request.args.get('first')
    last = request.args.get('last')
    # These go to where the get was used in a form and find which one had the label first or last, and grab the answer provided
    return render_template('thankyou.html', first=first, last=last)

@app.errorhandler(404)
def page_not_found(e):
    # The e is custom since it stands for error
    return render_template('404.html'), 404

if __name__ == '__main__':
    app.run(debug=True)

Let's demo our app! First we'll go to the home page and click the signup link. Then, we'll type our name into the form and click submit. After submitting, our app will redirect us to the thank you page and show our name.

![Home Page](img/lesson-3/demo-home.png)

![Signup Page](img/lesson-3/demo-signup.png)

![Thank You Page](img/lesson-3/demo-thanks.png)

## Advanced Forms

![SegmentLocal](https://media.tenor.com/images/11735777cd03fe010787526ce1704027/tenor.gif "segment")

In addition to creating forms in the `.html` files, we can use the `flask_wtf` and `WTForms` modules in our app. In this example, we'll call our Flask script `basic.py`.

To use `WTForms`, our `basic.py` will need to do a few more things than our last example:

- Configure a secret key for security
- Create a `WTForm` class instance with fields for each part of the form
- Create a view and allow `methods = ['GET', 'POST']`
- Create a `Form` class instance to handle a user submitting the form

Here is an example, with comments to break it down!

In [ ]:
# basic.py
from flask import Flask, render_template
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField

app = Flask(__name__)

# Setting configuration of secret key to an arbitrary string
app.config['SECRET_KEY'] = 'mysecretkey'

# InfoForm is the class of choice, inheriting from FlaskForm, which we imported from flask_wtf
class InfoForm(FlaskForm):
    # InfoForm has two attributes, the breed of the dog and a submit button
    breed = StringField("What breed are you?")
    submit = SubmitField('Submit')
    
# Allow get and post methods for this route (a post is when the form is submitted)
@app.route('/', methods=['GET', 'POST'])
def index():
    breed = False
    
    # Create a form
    form = InfoForm()
    
    # If the user submitted the form, and the form is valid
    if form.validate_on_submit():
        # Reset the form for more information to be submitted
        # But pass the old form data to the template to display
        breed = form.breed.data
        form.breed.data = ''

    return render_template('home.html', form=form, breed=breed)

if __name__ == '__main__':
    app.run(debug=True)

Our app has one view, `home.html`. If a form was submitted, the view will display the breed that the user `post`ed to the page. If no form was submitted (a `get` method), the view prompts the user to enter a breed.

![Prompt to Enter Breed](img/lesson-3/breed-prompt.png)

![Prompt to Enter Breed](img/lesson-3/breed-entering.png)

![Prompt to Enter Breed](img/lesson-3/breed-done.png)

### Form Fields

![SegmentLocal](https://media3.giphy.com/media/xUA7aLSKFAm4nmaeU8/giphy.gif "segment")

`WTForms` allows us to use any HTML form field type, such as checkboxes, radio buttons, and more. With `WTForms`, you can also _validate_ user inputs. Validating form data means verifying that the information submitted by a user suits what we're asking. For example, if we ask a user for a number, such as the number of lattes to order from Starbucks, we need to verify that the input is an integer, and not a string (`"One Grande Latte"`) or a float (`1.2` lattes). Validation can take on many other forms too, such as verifying that a user didn't submit blank information.

To demonstrate, let's add on to our dog breed example. We'll add a checkbox (or a `BooleanField` in `WTForms`), a radio button selection (or `RadioField`), a dropbox list (or a `SelectField`), and a large text area (or a `TextAreaField`). We'll also _validate_ our user's form input by ensuring that the breed (a `StringField`) exists via `DataRequired()`. 

In [ ]:
# basic.py

# Add importing session and redirect from the earlier example
from flask import Flask, render_template, session, redirect, url_for
from flask_wtf import FlaskForm

# The parentheses allow all the imports to be on seperate lines without error
from wtforms import (StringField, BooleanField, DateTimeField, RadioField,
                     SelectField, TextField, TextAreaField, SubmitField)

# Import DataRequired one of many possible form validators
from wtforms.validators import DataRequired


app = Flask(__name__)

# Set secret key to an arbitrary string
app.config['SECRET_KEY'] = 'mysecretkey'

class InfoForm(FlaskForm):
    # This is the first part of the form, a StringField, so users are presented with the question "What breed are you?" and then provide a string answer
    # DataRequired is used here as a validator, therefore it will make sure data is recieved for this question
    breed = StringField('What breed are you?', validators=[DataRequired()])

    # This is another form field, BooleanField, so the user is presented with the question, then can check the box for yes, or leave it unchecked for no (a checkbox)
    neutered = BooleanField("Have you been neutered?")

    # Another form field type, RadioField. For the choices, the first part, such as mood_one or mood_two, of each is for back_end
    # The second part, such as Happy or Excited, is what is presented to the user for each radio button, as the label
    mood = RadioField('Please choose your mood:', choices=[('mood_one', 'Happy'), ('mood_two', 'Excited')])

    # This works the same as the RadioField, but is presented in a different style than the radio buttons as a dropdown list
    food_choice = SelectField('Pick you favorite food:', choices=[('chi', 'Chicken'), ('bf', 'Beef'), ('fish', 'Fish')])

    # TextAreaField is for a user to write long-form (paragraph) text
    feedback = TextAreaField()

    # A SubmitField is a button for the user to submit the form
    submit = SubmitField('Submit')

@app.route('/', methods=['GET', 'POST'])
def index():
    form = InfoForm()
    if form.validate_on_submit():
        # This session dict is a way to store data for a specific amount of time, from when a user logs in to when they log out
        # Treat it like a dictionary, using 'breed' as a key
        session['breed'] = form.breed.data
        session['neutered'] = form.neutered.data
        session['mood'] = form.mood.data
        session['food'] = form.food_choice.data
        session['feedback'] = form.feedback.data

        # This redirection to the thankyou page only happens upon valid submission of the form
        return redirect(url_for('thankyou'))

    # Otherwise, return the index
    return render_template('home.html', form=form)

@app.route('/thankyou')
def thankyou():
    return render_template('thankyou.html')

if __name__ == '__main__':
    app.run(debug=True)


Add the following view `thankyou.html` in the `templates/` folder:

We see all of the expected form elements when vising `http://localhost:5000`:

![Puppy Form Index](img/lesson-3/breed2-blank.png)

And our `DataRequired()` validation ensures that a user needs to enter a breed when submitting the form:

![Puppy Form Index](img/lesson-3/breed2-error.png)

And upon successfully validating a submitted form, all of the information is displayed on the `thankyou.html` view: 

![Puppy Form Index](img/lesson-3/breed2-result.png)

### Toasts and Messages

![SegmentLocal](https://media.giphy.com/media/OQJQuRHDVJ1a8/giphy.gif "segment")

On web applications, there are many instances when you want to send a message to a user that you don't want to display permanently. For example, if a user submits a post on Facebook, you want to let the user know the submission was successful for a short time, then disappear.

Developers use toasts (or messages that stand out on a page) to display information to a user. Toasts can hide themselves after a period of time (also known as _flashing_ a message to a user) or wait until a user closes them. Here are some examples:

![SegmentLocal](https://javabeat.net/wp-content/uploads/2014/06/Bootstrap-Dismissal-Alert-Messages-Example.jpg "segment")

[Bootstrap](https://getbootstrap.com/) is a web framework that includes many built-in styles to use for web apps, including grids, form inputs, and toasts. (The first example of this lesson also used Bootstrap.)

Let's take a look at an example using Bootstrap to display toasts to a user when they click the Submit button:

In [ ]:
# basic.py

from flask import Flask, render_template, flash, session, redirect, url_for
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField

app = Flask(__name__)

app.config['SECRET_KEY'] = 'mykey'

class SimpleForm(FlaskForm):
    submit = SubmitField('Click Me')

@app.route('/', methods=['GET', 'POST'])
def index():
    form = SimpleForm()
    if form.validate_on_submit():
        flash('Holy guacamole! You just clicked the button!')
        return redirect(url_for('index'))

    return render_template('index.html', form=form)

if __name__ == '__main__':
    app.run(debug=True)

For this code, a button will appear on the home page, that says "Click Me"

To test our code, run the app and go to `http://localhost:5000`. There will be a button that says "Click Me":

![Click Me Button](img/lesson-3/click-me.png)

Clicking the button should display a dismissable blue toast, like the one shown here:

![Toast](img/lesson-3/click-me-toast.png)

And then you can dismiss the toast by clicking the "X". Nice job!

![SegmentLocal](https://media1.tenor.com/images/53285ed070aea4f963141d04a1d23fae/tenor.gif?itemid=7342741 "segment")

_Credit to Jose Portilla for code examples_

# Homework

Add forms to your flask application!